# USEFUL FOR PULLING ENVIRONMENTAL DATA ACROSS WHOLE REGIONS

In [10]:
import os
import geopandas as gpd
import pandas as pd
import ee
import geemap
import re
import time

In [2]:
# Authenticate to Earth Engine
ee.Authenticate()

True

In [2]:
# Authenticate to Earth Engine
#ee.Authenticate()
ee.Initialize()

In [11]:
class ElevationDownload:
    def __init__(self, elevation, folder_directory, data_folder, shp_dir):
        self.elevation = elevation
        self.folder_directory = folder_directory
        self.data_folder = data_folder
        self.shp_dir = shp_dir
        self.shp_files = [f for f in os.listdir(self.shp_dir) if f.endswith('.shp')]
        
    def process_shp_files(self, crs = "EPSG:4326", scale = "30"):
        count = 0
        # Iterate directory
        for path in os.listdir(self.shp_dir):
            # check if current path is a file
            if path.endswith('.shp'):
                count += 1
        print(f'Downloading data for {count} files')
        
        index_num = -1
        for shp_file in self.shp_files:
            regex_gcs = re.compile(r'[0-9]_gcs+')
            if (regex_gcs.search(shp_file) == None) == True:
                index_num = index_num + 1
                shp_path = os.path.join(self.shp_dir, shp_file)
                feature_collection = geemap.shp_to_ee(shp_path)
                feature = feature_collection.geometry()
                elv = self.elevation.filterBounds(feature).mosaic().reproject(crs).clip(feature).select(['elevation'])
                
                image_path = f'{self.folder_directory}/{self.data_folder}/cell_{index_num}_elevation_image.tif'
                geemap.ee_export_image(elv, image_path, scale, region = feature)
                print(f'Export task started for {shp_file}.')
                time.sleep(15)
            else:
                print("skipping gcs file")

        
        regex_gcs = re.compile(r'[0-9]_gcs+')
        delete_file_path = f'{self.shp_dir}/'
        delete_file_paths = os.listdir(delete_file_path)
        for file in delete_file_paths:
            full_path = f'{delete_file_path}{file}'
            if (regex_gcs.search(full_path) == None) == False:
                os.remove(full_path)
                print(f'removed {full_path}')

In [14]:
folder_directory = 'D:/!!Research/!!!Data/ArcGIS_Projects'
raster_working = f'{folder_directory}/buffers_tifs/raster_processing_tests'

# no slash needed at end of directory
shp_dir = f'{raster_working}/poly_fishnet_cells'

In [15]:
#shp_dir = f'{folder_directory}shp_file'
# Don't need to repeate the folder directory in an fstring, just need to tack on the
# additional directories that lead to the folder you want to store the data in.
# e.g. 'test_mesq_prediction_points/acc_points_env_data/env_data_no_ls'
data_folder = 'elevation_data'

# the 3DEP 1m collection
elevation = ee.ImageCollection('USGS/3DEP/1m')

# needs the overarching directory, the folder to put the data into, the stack of images to download, and
# the folder containing the polygon shapefiles
shp_data = ElevationDownload(elevation, raster_working, data_folder, shp_dir)
shp_data.process_shp_files(crs = 'EPSG:6350', scale = 1)

Generating URL ...
Please wait ...
Data downloaded to D:\!!Research\!!!Data\ArcGIS_Projects\buffers_tifs\raster_processing_tests\elevation_data\cell_0_elevation_image.tif
Export task started for fishnet_cell_0.shp.
Generating URL ...
Please wait ...
Data downloaded to D:\!!Research\!!!Data\ArcGIS_Projects\buffers_tifs\raster_processing_tests\elevation_data\cell_1_elevation_image.tif
Export task started for fishnet_cell_1.shp.
Generating URL ...
Please wait ...
Data downloaded to D:\!!Research\!!!Data\ArcGIS_Projects\buffers_tifs\raster_processing_tests\elevation_data\cell_2_elevation_image.tif
Export task started for fishnet_cell_10.shp.
Generating URL ...
Please wait ...
Data downloaded to D:\!!Research\!!!Data\ArcGIS_Projects\buffers_tifs\raster_processing_tests\elevation_data\cell_3_elevation_image.tif
Export task started for fishnet_cell_11.shp.
Generating URL ...
Please wait ...
Data downloaded to D:\!!Research\!!!Data\ArcGIS_Projects\buffers_tifs\raster_processing_tests\elevation